In [1]:
#lol y'a pas de module pour le torus on est dans la merde

In [5]:
# si k E Tp , il existe un entier r E [0;p] tq k = r/p
# on peut donc considérer k comme (r,p) ou juste comme r jsp

In [6]:
def AddInTore(p,a,b) :
    return (a+b)%p

In [18]:
def MultExtTore(p,n,x):
    if(n>=0):
        cpt = 0
        for i in range(n):
            cpt = AddInTore(p,cpt,x)
        return cpt
    return MultExtTore(p,-n,-x)

In [135]:
def TLWEncrypt(p,q,n,s,sigma,m):
    a=[]
    for i in range(n):
        a.append(randint(0,q-1))
    e = RealDistribution("gaussian",sigma).get_random_element()
    e = round(q*e)
    while(abs(e/q)>=1/(2*p)):
        e = RealDistribution("gaussian",sigma).get_random_element()
        e = round(q*e)
    #print("e= ",e)
    c = AddInTore(q,m*q/p,e)
    for i in range(n):
        c = AddInTore(q,c,MultExtTore(q,s[i],a[i]))
    return a+[c]

In [136]:
def TLWEDecrypt(p,q,n,s,c):
    a = c[:-1]
    m = c[-1]
    for i in range(n):
        m = AddInTore(q,m,MultExtTore(q,-s[i],a[i]))
    #print(m/q)
    return round(m*p/q)%p

In [141]:
p = 16
q = 64
n = 5
s = [0,1,1,0,0]
sigma = 0.001

m = 10

c = TLWEncrypt(p, q, n, s, sigma, m)
print("c =",c)

mmaybe = TLWEDecrypt(p, q, n, s, c)
print("m =",mmaybe)

c = [23, 31, 3, 2, 16, 10]
m = 10


In [168]:
p = 2
q = 2**32
n = 1000
s = [randint(0,1) for i in range(n)]
sigma = 2**(-7)

cpt = 0
for i in range(10000):
    
    m = randint(0,p-1)


    c = TLWEncrypt(p, q, n, s, sigma, m)

    mmaybe = TLWEDecrypt(p, q, n, s, c)
    if(m==mmaybe):
        cpt = cpt +1
print(cpt)        

10000
